- 전처리 미포함, 학습

In [1]:
import pandas as pd

review_jungrang_ori = pd.read_csv('C:/Users/user/Desktop/saltlux_project/ML_DL/preprocess/JungrangTotal.csv', index_col=0)
review_jungrang_ori = review_jungrang_ori[['name', 'nickname', 'content']].reset_index(drop=True)

In [2]:
review_jungrang_ori['content'] = review_jungrang_ori['content'].str.replace('\n', ' ')
review_jungrang_ori = review_jungrang_ori.reset_index(drop=True)
review_jungrang_ori

,name,nickname,content
0,365mc모인이비인후과의원,Wiseburge David,사가정 모인 이비인후과 항상 만원이다
1,365mc모인이비인후과의원,Dk9,좋아요
2,365mc모인이비인후과의원,귀한집딸z,좋아요
3,365mc모인이비인후과의원,꽃길걷는중임,항상 친절한 진료 감사합니다~
4,365mc모인이비인후과의원,막둥,ㅇㅇ
...,...,...,...
5970,효치과의원,fpvocalist,굳
5971,효치과의원,소보루39,좋아요
5972,효치과의원,1vvovv1,친절하십니다 과잉진료 없음
5973,효치과의원,버럭아저씨,친절해요..


In [10]:
import torch
import torch.nn as nn
from transformers import BertModel, AutoTokenizer
# from kobert_transformers import get_tokenizer
from torch.nn.functional import softmax
import pandas as pd

# 'content' 컬럼의 데이터를 리스트로 추출
texts = review_jungrang_ori['content'].tolist()

# KoBERT 모델 및 토크나이저 불러오기
model_name = 'skt/kobert-base-v1'
model = BertModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 추가 분류 레이어 정의
classifier = nn.Linear(768, 2)  # KoBERT의 hidden size는 768

# 결과를 저장할 리스트 초기화
results = []

# 임계값 설정
threshold = 0.4

# 가장 긴 텍스트의 길이
max_length = 512

# 각 텍스트에 대해 확률 계산 및 결과 저장
for text in texts:
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_length)

    with torch.no_grad():
        # 'last_hidden_state'를 사용하여 모델 출력 얻기
        outputs = model(**tokens)
    
    # 'last_hidden_state'에서 [CLS] 토큰에 해당하는 벡터 가져오기
    cls_token_embedding = outputs.last_hidden_state[:, 0, :]
    
    # 추가 분류 레이어를 통과시켜 로짓 계산
    logits = classifier(cls_token_embedding)
    
    # 확률로 변환
    probs = softmax(logits, dim=1).squeeze().tolist()
    
    # 임계값을 기준으로 긍정과 부정을 나눔
    result_str = f"긍정 확률: {probs[1]:.2%}" if probs[1] >= threshold else f"부정확률: {probs[0]:.2%}"
    results.append({'Text': text, 'Result': result_str})

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 결과를 CSV 파일로 저장
results_df.to_csv('kobert_results_ori_0.4prob.csv', index=False)


IndexError: index out of range in self